
# Capítulo 1 — Introdução à IA e ML

**Curso:** CECIERJ – IA e ML para Soluções Práticas  
**Objetivo do capítulo:** entender os conceitos fundamentais, os tipos de aprendizado e o *pipeline* completo de um projeto de ML, fechando com um mini-experimento guiado.

## O que é IA? E ML? E DL?
- **IA**: sistemas que realizam tarefas que normalmente exigem inteligência humana.
- **ML (Aprendizado de Máquina)**: subárea da IA que aprende *padrões a partir de dados*.
- **DL (Aprendizado Profundo)**: subárea de ML baseada em redes neurais com muitas camadas.

> Resumo: **IA ⊇ ML ⊇ DL** — nem toda IA usa ML; nem todo ML é DL.



## Tipos de Aprendizado
- **Supervisionado**: dados com rótulo/valor alvo (ex.: classificar spam, prever preço).
- **Não supervisionado**: dados sem rótulo (ex.: agrupar clientes, reduzir dimensionalidade).
- **Por reforço**: agente aprende por *recompensas* em um ambiente (ex.: jogos, robótica).



## Pipeline de um Projeto de ML
1. **Definir problema e objetivos** (o que otimizar? qual métrica importa?).  
2. **Entender/coletar dados** (EDA, qualidade, viés).  
3. **Pré-processar/engenharia de atributos** (limpeza, *encoding*, escalonamento).  
4. **Treinar e validar** (holdout e/ou *cross-validation*).  
5. **Ajustar hiperparâmetros** (ex.: `GridSearchCV`).  
6. **Avaliar** (métricas + interpretação de resultados).  
7. **Empacotar e versionar** (reprodutibilidade).  
8. **Implantar e monitorar** (drift, performance, custo).  
9. **Ética e governança** (LGPD, vieses, transparência).



## Métricas (mapa rápido)
- **Classificação**: *accuracy*, *precision*, *recall*, *F1*, *ROC AUC*.
- **Regressão**: *MAE*, *MSE*, *RMSE*, *R²*.
- **Clustering**: *silhouette*, *inertia* (KMeans).

> Não existe “uma métrica para tudo”. Selecione de acordo com o problema.



## Riscos comuns
- **Overfitting/Underfitting**: ajuste demais/de menos.
- **Data leakage**: usar acidentalmente informação do futuro no treino.
- **Validação incorreta**: sem CV, resultados instáveis.
- **Métrica desalinhada**: otimiza *accuracy*, mas o custo real depende de *recall*.



---
# Mini‑experimento: Classificação binária (do zero em poucos passos)

Vamos prever se um tumor é **maligno** (1) ou **benigno** (0) usando o dataset **Breast Cancer** do `scikit-learn`.  
Etapas: carregar dados → *train/test split* → *pipeline* (escalonamento + modelo) → avaliação.


In [ ]:

# Checagem de ambiente (execute para confirmar versões)
import sys
print("Python:", sys.version)
try:
    import numpy, pandas, sklearn, matplotlib
    print("numpy:", numpy.__version__)
    print("pandas:", pandas.__version__)
    print("scikit-learn:", sklearn.__version__)
    print("matplotlib:", matplotlib.__version__)
except Exception as e:
    print("Dependências ausentes:", e)


In [ ]:

# 1) Carregar dados e olhar formato básico
import pandas as pd
from sklearn.datasets import load_breast_cancer

ds = load_breast_cancer(as_frame=True)
df = ds.frame.copy()  # contém features + coluna target
print("Shape:", df.shape)
df.head()


In [ ]:

# 2) Split + Pipeline (StandardScaler + LogisticRegression)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

X = df.drop(columns=["target"])
y = df["target"]  # 1 = maligno, 0 = benigno

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])

pipe.fit(X_train, y_train)
print("Treinado!")


In [ ]:

# 3) Avaliação rápida: accuracy, relatório de classificação,
#    matriz de confusão e curva ROC
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, RocCurveDisplay
)

y_pred = pipe.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)
print(classification_report(y_test, y_pred, digits=4))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay(cm).plot()
plt.title("Matriz de Confusão")
plt.show()

# Curva ROC (binário)
RocCurveDisplay.from_estimator(pipe, X_test, y_test)
plt.title("Curva ROC")
plt.show()


In [ ]:

# 4) Interpretação rápida: coeficientes (importância aproximada no modelo linear)
import numpy as np
clf = pipe.named_steps["clf"]
coefs = pd.Series(clf.coef_[0], index=X.columns).sort_values(ascending=False)
coefs.head(10)



## Conclusões e próximos passos
- Você viu o *pipeline* mínimo: **split → scaler → modelo → avaliação**.
- A partir daqui (demais capítulos), aprofundamos **pré-processamento**, **validação**, **tuning** e **métricas**.
- Pense na **métrica de negócio** antes de otimizar o modelo.

> Dica: sempre guarde **seeds**, **versões** e **scripts** para reprodutibilidade.
